In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from utilities.preprocessors import column_summary 
from utilities.visualizers import disp_cat_feat, view_feat_outliers

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
DATA_DIR = './data/population-data'
EXCLUSIONS = ["us_populations_per_state_2001_to_2021.csv"]
files = list(filter(lambda file: not file in EXCLUSIONS, os.listdir(DATA_DIR)))
len(files)

152

In [18]:
test_df = pd.read_excel(os.path.join(DATA_DIR, "Alabama_2000-2010.xls"), dtype=object, header=None)
test_df.head(40)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,table with row headers in column A and column ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Table 2. Intercensal Estimates of the Resident...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sex and Age,"April 1, 20001",Intercensal Estimates (as of July 1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"April 1, 20102","July 1, 20103"
3,NaN,NaN,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,NaN,NaN
4,BOTH SEXES,4447207,4452173,4467634,4480089,4503491,4530729,4569805,4628981,4672840,4718206,4757938,4779736,4785298
5,.Under 5 years,296000,295185,296624,296046,295204,295970,296441,297222,300300,304842,305412,304957,304840
6,.5 to 9 years,315369,313178,307526,302632,299148,297554,298450,303581,306013,306682,307864,308229,308125
7,.10 to 14 years,320266,321372,323615,325008,326642,326228,323028,321867,320407,319503,319072,319655,319314
8,.15 to 19 years,324583,325612,321866,320749,321655,325095,330753,337003,341279,345580,346611,343471,341504
9,.20 to 24 years,306876,309170,318741,322812,326983,326749,326727,326239,327293,328751,332117,335322,336601


#### These are the age brackets of these dataframes what we want to do is transform this dataframe into this

In [25]:
test_df.loc[:40, 0].to_list()

['table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)',
 'Table 2. Intercensal Estimates of the Resident Population by Sex and Age for Alabama: April 1, 2000 to July 1, 2010',
 'Sex and Age',
 nan,
 'BOTH SEXES',
 '.Under 5 years',
 '.5 to 9 years',
 '.10 to 14 years',
 '.15 to 19 years',
 '.20 to 24 years',
 '.25 to 29 years',
 '.30 to 34 years',
 '.35 to 39 years',
 '.40 to 44 years',
 '.45 to 49 years',
 '.50 to 54 years',
 '.55 to 59 years',
 '.60 to 64 years',
 '.65 to 69 years',
 '.70 to 74 years',
 '.75 to 79 years',
 '.80 to 84 years',
 '.85 years and over',
 '.',
 '.Under 18 years',
 '.Under 5 years',
 '.5 to 13 years',
 '.14 to 17 years',
 '.18 to 64 years',
 '.18 to 24 years',
 '.25 to 44 years',
 '.45 to 64 years',
 '.65 years and over',
 '.',
 '.16 years and over',
 '.18 years and over',
 '.15 to 44 years',
 '.',
 '.Median age (years)',
 'MALE',
 '.Under 5 years']

#### Next is to extract out the unique age brackets and use those as columns in the table/dataframe we will be modelling together with its dimensions and all

In [12]:
for file in files[:3]:
    print(file)
    FILE_PATH = os.path.join(DATA_DIR, file)
    df = pd.read_excel(FILE_PATH)
    print(df)

Alabama_2000-2010.xls
    table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)  \
0    Table 2. Intercensal Estimates of the Resident...                                                             
1                                          Sex and Age                                                             
2                                                  NaN                                                             
3                                           BOTH SEXES                                                             
4                                       .Under 5 years                                                             
..                                                 ...                                                             
111  Note: Median age is calculated based on single...                                                             
112                                Suggested Citat